# Preprocess the data

This script preprocesses data from Stack Overflow Developer Survey.

- download the data from https://survey.stackoverflow.co/ if not already downloaded
- extract relevant columns, convert them to uniform format
- clean data: remove outliers by salary, years of experience, age, etc..
- save into `src/assets/data.csv` from where the frontend will load it


In [18]:
import os
import zipfile
import requests
from pathlib import Path
import pandas as pd

# URLs for the .zip files by year
urls_by_year = {
    "2017": "https://survey.stackoverflow.co/datasets/stack-overflow-developer-survey-2017.zip",
    "2018": "https://info.stackoverflowsolutions.com/rs/719-EMH-566/images/stack-overflow-developer-survey-2018.zip",
    "2019": "https://info.stackoverflowsolutions.com/rs/719-EMH-566/images/stack-overflow-developer-survey-2019.zip",
    "2020": "https://survey.stackoverflow.co/datasets/stack-overflow-developer-survey-2020.zip",
    "2021": "https://info.stackoverflowsolutions.com/rs/719-EMH-566/images/stack-overflow-developer-survey-2021.zip",
    "2022": "https://info.stackoverflowsolutions.com/rs/719-EMH-566/images/stack-overflow-developer-survey-2022.zip",
    "2023": "https://cdn.stackoverflow.co/files/jo7n4k8s/production/49915bfd46d0902c3564fd9a06b509d08a20488c.zip",
    "2024": "https://cdn.sanity.io/files/jo7n4k8s/production/262f04c41d99fea692e0125c342e446782233fe4.zip",
}

# Directory to store raw data
raw_data_dir = Path("data") / "raw"

# Ensure the directory exists
raw_data_dir.mkdir(parents=True, exist_ok=True)

def download_and_extract(year: str, url: str) -> Path:
    save_path = raw_data_dir / f"survey-{year}"
    if save_path.exists():
        # Already downloaded
        print(f"Skipping {year} data download, already exists")
        return save_path / "survey_results_public.csv"

    # Create the zip file path
    zip_file_path = raw_data_dir / f"stack-overflow-developer-survey-{year}.zip"

    # Download the .zip file
    print(f"Downloading {year} data...")
    response = requests.get(url)
    with open(zip_file_path, "wb") as file:
        file.write(response.content)

    with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
        zip_ref.extractall(save_path)

    # Clean up the zip file after extraction
    os.remove(zip_file_path)

    # Return the path to the CSV file inside the extracted folder
    return save_path / "survey_results_public.csv"

# Download and extract the data, then read it into pandas DataFrame
dfs_by_year = {}
for year, url in urls_by_year.items():
    csv_path = download_and_extract(year, url)
    dfs_by_year[year] = pd.read_csv(csv_path)


Skipping 2017 data download, already exists
Skipping 2018 data download, already exists


/var/folders/y5/_fnn1tf53cz1df_1wxhgm5m00000gn/T/ipykernel_44905/1358424381.py:54: DtypeWarning: Columns (8,12,13,14,15,16,50,51,52,53,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs_by_year[year] = pd.read_csv(csv_path)


Skipping 2019 data download, already exists
Skipping 2020 data download, already exists
Skipping 2021 data download, already exists
Skipping 2022 data download, already exists
Skipping 2023 data download, already exists
Skipping 2024 data download, already exists


## unify the datasets

- rename columns to uniform names
- convert years/age ranges (e.g. "20 to 30 years old") to values (using interval midpoints)
- remove rows that have missing values for salary, programming langs, country or years of experience
- approximate age using years of experience if the column is missing (in the 2017 survey)

In [19]:
from enum import Enum

# These columns are used in the frontend
class Column(Enum):
    SALARY = "salary"
    COUNTRY = "country"
    COUNTRY_CODE = "country_code"
    YEARS_OF_EXPERIENCE = "years_of_experience"
    AGE = "age"
    PROGRAMMING_LANGUAGE = "language"

In [20]:
column_map_2021_and_later = {
    Column.SALARY: "ConvertedCompYearly",
    Column.COUNTRY: "Country",
    Column.COUNTRY_CODE: None,
    Column.YEARS_OF_EXPERIENCE: "YearsCode",
    Column.AGE: "Age",
    Column.PROGRAMMING_LANGUAGE: "LanguageHaveWorkedWith",
}

colum_map_by_year = {
    "2017": {
        Column.SALARY: "Salary",
        Column.COUNTRY: "Country",
        Column.COUNTRY_CODE: None,
        Column.YEARS_OF_EXPERIENCE: "YearsProgram",
        Column.AGE: None,  # Approximated YearsProgram + 23
        Column.PROGRAMMING_LANGUAGE: "HaveWorkedLanguage"
    },
    "2018": {
        Column.SALARY: "ConvertedSalary",
        Column.COUNTRY: "Country",
        Column.COUNTRY_CODE: None,
        Column.YEARS_OF_EXPERIENCE: "YearsCoding",
        Column.AGE: "Age",
        Column.PROGRAMMING_LANGUAGE: "LanguageWorkedWith",
    },
    "2019": {
        Column.SALARY: "ConvertedComp",
        Column.COUNTRY: "Country",
        Column.COUNTRY_CODE: None,
        Column.YEARS_OF_EXPERIENCE: "YearsCode",
        Column.AGE: "Age",
        Column.PROGRAMMING_LANGUAGE: "LanguageWorkedWith",
    },
    "2020": {
        Column.SALARY: "ConvertedComp",
        Column.COUNTRY: "Country",
        Column.COUNTRY_CODE: None,
        Column.YEARS_OF_EXPERIENCE: "YearsCode",
        Column.AGE: "Age",
        Column.PROGRAMMING_LANGUAGE: "LanguageWorkedWith",
    },
    "2021": column_map_2021_and_later,
    "2022": column_map_2021_and_later,
    "2023": column_map_2021_and_later,
    "2024": column_map_2021_and_later,
}

### Convert years/age ranges to values

In [21]:
import re

def convert_years_and_age(df: pd.DataFrame):
    def convert_to_numeric(value):
        if pd.isnull(value):
            return None
        if isinstance(value, (int, float)):  # If already a number, return it
            return float(value)
        value = str(value)

        # Match patterns and convert accordingly
        if re.match(r'^\d+$', value):
            return float(value)
        elif match := re.match(r'^(\d+)\s*years?', value):
            return float(match.group(1))
        elif match := re.match(r'^(\d+)\s*to\s*(\d+)', value):
            return (float(match.group(1)) + float(match.group(2))) / 2
        elif match := re.match(r'^(\d+)[\s\-]+(\d+)', value):
            return (float(match.group(1)) + float(match.group(2))) / 2
        elif match := re.match(r'^(\d+)\s*and\s*more', value):
            return float(match.group(1)) + 1
        elif match := re.match(r'^(\d+)\s*or\s*more', value):
            return float(match.group(1)) + 1
        else:
            return None

    # Apply conversion to specified columns
    df[Column.YEARS_OF_EXPERIENCE.value] = df[Column.YEARS_OF_EXPERIENCE.value].apply(convert_to_numeric)
    df[Column.AGE.value] = df[Column.AGE.value].apply(convert_to_numeric)

    return df

### Extract uniform country codes

In [22]:
import pycountry
import country_converter as coco
import logging

country_converter = coco.CountryConverter(include_obsolete=True)

# Do not log warnings from coco (if match is not found)
logging.basicConfig(level=logging.ERROR, force=True)

# Function to get ISO Alpha-2 code
def get_country_code(country_name: str):
    try:
        # Faster but less robust pycountry matcher
        country = pycountry.countries.lookup(country_name.strip())
        return country.alpha_2
    except LookupError:
        # Fallback to coco which is more robust but slower
        country = country_converter.convert(names=country_name, to='ISO2', not_found='NOT-FOUND')
        return None if country == 'NOT-FOUND' else country


### Apply everything on the datasets and save the results

In [23]:
save_path = Path("data/extracted")
os.makedirs(save_path, exist_ok=True)

# Do not log warnings from coco (if match is not found)
logging.basicConfig(level=logging.ERROR, force=True)

dfs_normalized_by_year = {}

for year, df in dfs_by_year.items():
    if year not in colum_map_by_year:
        continue

    # Extract relevant columns
    column_map = colum_map_by_year[year]
    columns = [column_map[col] for col in Column if column_map[col] is not None]
    df = df[columns]

    # Rename columns
    df.columns = [col.value for col in Column if column_map[col] is not None]

    # Remove rows that don't have salary, language, country or years of experience
    df = df.dropna(subset=[Column.SALARY.value, Column.PROGRAMMING_LANGUAGE.value, Column.COUNTRY.value, Column.YEARS_OF_EXPERIENCE.value])


    # Year 2017 does not have age column, set values to None
    if year == "2017":
        df[Column.AGE.value] = None

    # Convert y.o.e and age from range to numeric
    df = convert_years_and_age(df)
    print(f"Extracted {len(df)} rows for year {year}")

    # Approximate age with years of experience + 23 if it's nan
    df[Column.AGE.value] = df[Column.AGE.value].fillna(df[Column.YEARS_OF_EXPERIENCE.value] + 23)

    # Standardize country names e.g. [USA, United States, ...] to country codes (US)
    df[Column.COUNTRY_CODE.value] = df[Column.COUNTRY.value].apply(get_country_code)

    # Remove rows with no country code (couldn't be matched, there's just 10-20 of them)
    df = df.dropna(subset=[Column.COUNTRY_CODE.value])

    df.to_csv(save_path / f"{year}.csv", index=False)
    dfs_normalized_by_year[year] = df


Extracted 12120 rows for year 2017


/var/folders/y5/_fnn1tf53cz1df_1wxhgm5m00000gn/T/ipykernel_44905/2029844218.py:34: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[Column.AGE.value] = df[Column.AGE.value].fillna(df[Column.YEARS_OF_EXPERIENCE.value] + 23)


Extracted 46467 rows for year 2018
Extracted 55537 rows for year 2019
Extracted 33333 rows for year 2020
Extracted 46329 rows for year 2021
Extracted 37891 rows for year 2022
Extracted 47820 rows for year 2023
Extracted 23309 rows for year 2024


## Extend over language column

now the `language` column contains multiple programming languages concat'd by a delimiter e.g. "Python; R; SQL". We will split the rows into multiple rows, each containing a single language.

In [24]:
from typing import Optional


def parse_language(lang: str) -> Optional[str]:
    lang = lang.strip()

    if lang in ["CSS", "SQL", "SQL Server", "MongoDB", "Node.js"]:
        # Let's be real, these are not real programming languages
        return None

    if "bash" in lang.lower() or "html" in lang.lower():
        return None
    
    return lang

def expand_by_language(df: pd.DataFrame):
    # Create an empty list to store the expanded rows
    expanded_rows = []

    # Iterate over the rows of the dataframe
    for _, row in df.iterrows():
        # Split the 'language' column by ';'
        languages = row[Column.PROGRAMMING_LANGUAGE.value].split(';')

        # For each language, create a new row with the same values but different language
        for language in languages:
            language = parse_language(language)
            if language is None:
                continue

            expanded_row = row.copy() 
            expanded_row[Column.PROGRAMMING_LANGUAGE.value] = language
            expanded_rows.append(expanded_row)

    # Convert the list of expanded rows back into a DataFrame
    expanded_df = pd.DataFrame(expanded_rows)

    return expanded_df

In [25]:
save_path = Path("data/expanded")
os.makedirs(save_path, exist_ok=True)

expanded_df_by_year = {}


for year, df in dfs_normalized_by_year.items():
    df_expanded = expand_by_language(df)

    print(f"{year}: {len(df)} rows expanded to {len(df_expanded)} rows")


    expanded_df_by_year[year] = df_expanded

    df_expanded.to_csv(save_path / f"{year}.csv", index=False)



2017: 12107 rows expanded to 38152 rows
2018: 46428 rows expanded to 180642 rows
2019: 55477 rows expanded to 189266 rows
2020: 33319 rows expanded to 113856 rows
2021: 46311 rows expanded to 166636 rows
2022: 37875 rows expanded to 137698 rows
2023: 47803 rows expanded to 180633 rows
2024: 23305 rows expanded to 90008 rows


## Subsets the data for top `n` languages

- since there are many too many unique programming languages visualizing all of them would result in cluttered graphs
- subset only the top `n` most popular languages

In [26]:
from collections import Counter

# Initialize a Counter to accumulate language occurrences across all years
language_counter = Counter()

for year, df in expanded_df_by_year.items():
    # Count the occurrences of each language in the expanded dataframe
    language_counts = df[Column.PROGRAMMING_LANGUAGE.value].value_counts()
    # Update the Counter with the language counts as a dictionary
    language_counter.update(language_counts.to_dict())

# After processing all years, print the overall language counts sorted by occurrence
sorted_language_counts = language_counter.most_common()

n = 20
top_n_languages = sorted_language_counts[:n]

# Print the top n languages
for language, count in top_n_languages:
    print(f"{language}: {count}")


JavaScript: 206159
Python: 126607
Java: 103976
TypeScript: 96040
C#: 95326
PHP: 68739
C++: 57007
C: 46979
Go: 32587
Ruby: 25967
Kotlin: 22364
PowerShell: 20826
Rust: 18854
Swift: 17960
R: 14143
VBA: 13787
Objective-C: 12538
Assembly: 12199
Scala: 11301
Dart: 10251


Remove all irrelevant rows from the dataframes (ones that are not related to top 20 programming languages by popularity)

In [27]:

save_path = Path("data/cleaned")
os.makedirs(save_path, exist_ok=True)

top_languages_set = {lang for lang, _ in top_n_languages}

df_clean_by_year = {}

total_rows = 0

for year, df in expanded_df_by_year.items():
    # Filter out rows with languages that are not in the top n languages
    df_clean = df[df[Column.PROGRAMMING_LANGUAGE.value].isin(top_languages_set)]

    df_clean = df_clean.dropna()
    print(f"{year}: {len(df)} rows filtered to {len(df_clean)} rows")
    total_rows += len(df_clean)

    df_clean['year'] = year
    df_clean_by_year[year] = df_clean
    df_clean.to_csv(save_path / f"{year}.csv", index=False)

print(f"Total rows: {total_rows}")

2017: 38152 rows filtered to 34219 rows
2018: 180642 rows filtered to 160821 rows
2019: 189266 rows filtered to 180275 rows
2020: 113856 rows filtered to 111638 rows
2021: 166636 rows filtered to 155463 rows
2022: 137698 rows filtered to 126205 rows
2023: 180633 rows filtered to 161740 rows
2024: 90008 rows filtered to 80691 rows
Total rows: 1011052


## Filter outliers by IQR

Some of the records are very visible outliers (e.g. a few records of 10M+ salary in Venezuela while the average of the rest is 10k). We can remove them by filtering with IQR (interquartile range) which is a measure of the spread of the data. Data points that are more than 1.5 times the IQR away from the median are considered outliers.

- remove outliers by salary
- remove outliers by years of experience
- remove outliers by age


In [28]:
min_rows_for_group = 10

def iqr_mask(series):
    q1, q3 = series.quantile([0.25, 0.75])
    iqr = q3 - q1
    return series.between(q1 - 1.5*iqr, q3 + 1.5*iqr)

for year, df in df_clean_by_year.items():
    original_size = len(df)
    grouping_cols = [Column.COUNTRY.value, Column.PROGRAMMING_LANGUAGE.value]

    # Define a function to filter a single group
    def filter_group(group):
        # Apply filtering conditions on the group
        filtered = group[
            iqr_mask(group[Column.SALARY.value]) &
            iqr_mask(group[Column.YEARS_OF_EXPERIENCE.value]) &
            iqr_mask(group[Column.AGE.value]) &
            group[Column.SALARY.value].between(501, 499_999)
        ]
        # Return the filtered group if it meets the row threshold
        return filtered if len(filtered) >= min_rows_for_group else pd.DataFrame()

    # Apply the filtering function to each group and retain the results
    filtered_groups = [
        filter_group(group) for _, group in df.groupby(grouping_cols) if not group.empty
    ]

    # Combine all the filtered groups back into a single DataFrame
    df_clean = pd.concat(filtered_groups, axis=0).reset_index(drop=True) if filtered_groups else pd.DataFrame()

    print(f"{year}: {original_size} -> {len(df_clean)} rows")
    df_clean_by_year[year] = df_clean



2017: 34219 -> 30006 rows
2018: 160821 -> 126298 rows
2019: 180275 -> 150503 rows
2020: 111638 -> 93323 rows
2021: 155463 -> 126913 rows
2022: 126205 -> 101068 rows
2023: 161740 -> 138667 rows
2024: 80691 -> 66682 rows


## Merge all survey years into one dataframe

In [29]:
# Save into src/assets/data.csv from where the frontend will load it

save_path = Path("src") / "assets"

df_all = pd.concat(df_clean_by_year.values(), ignore_index=True)
df_all.to_csv(save_path / "data.csv", index=False)
